In [ ]:
%pip install -r requirements.txt

In [29]:
import requests
import re
import urllib.request
from bs4 import BeautifulSoup
from collections import deque
from html.parser import HTMLParser
from urllib.parse import urlparse
import os

# 匹配URL的正则表达式模式
HTTP_URL_PATTERN = r'^http[s]*://.+'

# 定义要抓取的网址
domain = "deepseek.com"
full_url = "https://platform.deepseek.com/api-docs/zh-cn/"

# 创建一个类来解析HTML并获取超链接
class HyperlinkParser(HTMLParser):
    def __init__(self):
        super().__init__()
        # 创建一个数组来存储超链接
        self.hyperlinks = []

    # 重写HTMLParser的handle_starttag方法以获取超链接
    def handle_starttag(self, tag, attrs):
        attrs = dict(attrs)

        # 如果标签是锚标签并且具有href属性，则将href属性添加到超链接列表中
        if tag == "a" and "href" in attrs:
            self.hyperlinks.append(attrs["href"])

# 从URL获取超链接的函数
def get_hyperlinks(url):
    
    # 尝试打开URL并阅读超文本标记语言
    try:
        # 打开URL并阅读内容
        with urllib.request.urlopen(url) as response:

            # 如果响应不是html，则返回一个空列表
            if not response.info().get('Content-Type').startswith("text/html"):
                return []
            
            # 解码html
            html = response.read().decode('utf-8')
    except Exception as e:
        print(e)
        return []

    # 创建html解析器，然后解析html以获取超链接
    parser = HyperlinkParser()
    parser.feed(html)

    return parser.hyperlinks

# 从同一域中的URL获取超链接的函数
def get_domain_hyperlinks(local_domain, url):
    clean_links = []
    for link in set(get_hyperlinks(url)):
        clean_link = None

        # 如果链接是URL，检查它是否在同一域内
        if re.search(HTTP_URL_PATTERN, link):
            # 解析URL并检查域是否相同
            url_obj = urlparse(link)
            if url_obj.netloc == local_domain:
                clean_link = link

        # 如果链接不是URL，请检查它是否是相对链接
        else:
            if link.startswith("/"):
                link = link[1:]
            elif link.startswith("#") or link.startswith("mailto:"):
                continue
            clean_link = "https://" + local_domain + "/" + link

        if clean_link is not None:
            if clean_link.endswith("/"):
                clean_link = clean_link[:-1]
            clean_links.append(clean_link)

    # 返回同一域中的超链接列表
    return list(set(clean_links))


def crawl(url):
    # 解析URL并获取域
    local_domain = urlparse(url).netloc

    # 创建一个队列来存储要抓取的URL
    queue = deque([url])

    # 创建一个集合来存储已经看到的URL（没有重复）
    seen = set([url])

    # 创建一个目录来存储文本文件
    if not os.path.exists("text/"):
            os.mkdir("text/")

    if not os.path.exists("text/"+local_domain+"/"):
            os.mkdir("text/" + local_domain + "/")

    # 创建一个目录来存储csv文件
    if not os.path.exists("processed"):
            os.mkdir("processed")

    # 当队列不为空时，继续
    while queue:

        # 从队列中获取下一个URL
        url = queue.pop()
        print(url)

        # 将文本从url保存到<url>. txt文件
        with open('text/'+local_domain+'/'+url[8:].replace("/", "_") + ".txt", "w", encoding="utf-8") as f:

            # 使用BeautifulSoup从URL获取文本
            soup = BeautifulSoup(requests.get(url).text, "html.parser")

            # 获取文本但删除标签
            text = soup.get_text()

            # 如果爬虫到达需要JavaScript的页面，它将停止爬行
            if ("You need to enable JavaScript to run this app." in text):
                print("Unable to parse page " + url + " due to JavaScript being required")
            
            # 否则，将文本写入文本目录中的文件
            f.write(text)

        # 从URL获取超链接并将它们添加到队列中
        for link in get_domain_hyperlinks(local_domain, url):
            if link not in seen:
                queue.append(link)
                seen.add(link)

crawl(full_url)

https://platform.deepseek.com/api-docs/zh-cn/
https://platform.deepseek.com/api-docs/zh-cn/api/create-chat-completion
https://platform.deepseek.com/api-docs/api/create-chat-completion
https://platform.deepseek.com/api-docs/api/deepseek-api
https://platform.deepseek.com/downloads/DeepSeek%20Open%20Platform%20Terms%20of%20Service.html
https://platform.deepseek.com/api-docs/api/get-user-balance
https://platform.deepseek.com/api-docs/pricing
https://platform.deepseek.com/api-docs/api/list-models
https://platform.deepseek.com/api-docs/faq
https://platform.deepseek.com/transactions
https://platform.deepseek.com/api-docs/updates
https://platform.deepseek.com/api-docs
https://platform.deepseek.com/top_up
https://platform.deepseek.com/api-docs/zh-cn/api/get-user-balance
https://platform.deepseek.com/api_keys
https://platform.deepseek.com/api-docs/zh-cn/updates
https://platform.deepseek.com/api-docs/zh-cn
https://platform.deepseek.com//top_up
https://platform.deepseek.com/api-docs/zh-cn/api/list

In [ ]:
import requests
import re
import urllib.request
from bs4 import BeautifulSoup
from collections import deque
from html.parser import HTMLParser
from urllib.parse import urlparse
import os

# 匹配URL的正则表达式模式
HTTP_URL_PATTERN = r'^http[s]*://.+'

# 定义要抓取的网址
domain = "openai.com"
full_url = "https://openai.com/"

# 创建一个类来解析HTML并获取超链接
class HyperlinkParser(HTMLParser):
    def __init__(self):
        super().__init__()
        # 创建一个数组来存储超链接
        self.hyperlinks = []

    # 重写HTMLParser的handle_starttag方法以获取超链接
    def handle_starttag(self, tag, attrs):
        attrs = dict(attrs)

        # 如果标签是锚标签并且具有href属性，则将href属性添加到超链接列表中
        if tag == "a" and "href" in attrs:
            self.hyperlinks.append(attrs["href"])

# 从URL获取超链接的函数
def get_hyperlinks(url):
    
    # 尝试打开URL并阅读超文本标记语言
    try:
        # 打开URL并阅读内容
        with urllib.request.urlopen(url) as response:

            # 如果响应不是html，则返回一个空列表
            if not response.info().get('Content-Type').startswith("text/html"):
                return []
            
            # 解码html
            html = response.read().decode('utf-8')
    except Exception as e:
        print(e)
        return []

    # 创建html解析器，然后解析html以获取超链接
    parser = HyperlinkParser()
    parser.feed(html)

    return parser.hyperlinks

# 从同一域中的URL获取超链接的函数
def get_domain_hyperlinks(local_domain, url):
    clean_links = []
    for link in set(get_hyperlinks(url)):
        clean_link = None

        # 如果链接是URL，检查它是否在同一域内
        if re.search(HTTP_URL_PATTERN, link):
            # 解析URL并检查域是否相同
            url_obj = urlparse(link)
            if url_obj.netloc == local_domain:
                clean_link = link

        # 如果链接不是URL，请检查它是否是相对链接
        else:
            if link.startswith("/"):
                link = link[1:]
            elif link.startswith("#") or link.startswith("mailto:"):
                continue
            clean_link = "https://" + local_domain + "/" + link

        if clean_link is not None:
            if clean_link.endswith("/"):
                clean_link = clean_link[:-1]
            clean_links.append(clean_link)

    # 返回同一域中的超链接列表
    return list(set(clean_links))


def crawl(url):
    # 解析URL并获取域
    local_domain = urlparse(url).netloc

    # 创建一个队列来存储要抓取的URL
    queue = deque([url])

    # 创建一个集合来存储已经看到的URL（没有重复）
    seen = set([url])

    # 创建一个目录来存储文本文件
    if not os.path.exists("text/"):
            os.mkdir("text/")

    if not os.path.exists("text/"+local_domain+"/"):
            os.mkdir("text/" + local_domain + "/")

    # 创建一个目录来存储csv文件
    if not os.path.exists("processed"):
            os.mkdir("processed")

    # 当队列不为空时，继续
    while queue:

        # 从队列中获取下一个URL
        url = queue.pop()
        print(url)

        # 将文本从url保存到<url>. txt文件
        with open('text/'+local_domain+'/'+url[8:].replace("/", "_") + ".txt", "w", encoding="utf-8") as f:

            # 使用BeautifulSoup从URL获取文本
            soup = BeautifulSoup(requests.get(url).text, "html.parser")

            # 获取文本但删除标签
            text = soup.get_text()

            # 如果爬虫到达需要JavaScript的页面，它将停止爬行
            if ("You need to enable JavaScript to run this app." in text):
                print("Unable to parse page " + url + " due to JavaScript being required")
            
            # 否则，将文本写入文本目录中的文件
            f.write(text)

        # 从URL获取超链接并将它们添加到队列中
        for link in get_domain_hyperlinks(local_domain, url):
            if link not in seen:
                queue.append(link)
                seen.add(link)

crawl(full_url)

https://openai.com/
HTTP Error 403: Forbidden


In [ ]:
import requests
import re
import urllib.request
from bs4 import BeautifulSoup
from collections import deque
from html.parser import HTMLParser
from urllib.parse import urlparse
import os

# 匹配URL的正则表达式模式
HTTP_URL_PATTERN = r'^http[s]*://.+'

# 定义要抓取的网址
domain = "openai.com"
full_url = "https://openai.com/"

# 创建一个类来解析HTML并获取超链接
class HyperlinkParser(HTMLParser):
    def __init__(self):
        super().__init__()
        # 创建一个数组来存储超链接
        self.hyperlinks = []

    # 重写HTMLParser的handle_starttag方法以获取超链接
    def handle_starttag(self, tag, attrs):
        attrs = dict(attrs)

        # 如果标签是锚标签并且具有href属性，则将href属性添加到超链接列表中
        if tag == "a" and "href" in attrs:
            self.hyperlinks.append(attrs["href"])

# 从URL获取超链接的函数
def get_hyperlinks(url):
    
    # 尝试打开URL并阅读超文本标记语言
    try:
        # 打开URL并阅读内容
        with urllib.request.urlopen(url) as response:

            # 如果响应不是html，则返回一个空列表
            if not response.info().get('Content-Type').startswith("text/html"):
                return []
            
            # 解码html
            html = response.read().decode('utf-8')
    except Exception as e:
        print(e)
        return []

    # 创建html解析器，然后解析html以获取超链接
    parser = HyperlinkParser()
    parser.feed(html)

    return parser.hyperlinks

# 从同一域中的URL获取超链接的函数
def get_domain_hyperlinks(local_domain, url):
    clean_links = []
    for link in set(get_hyperlinks(url)):
        clean_link = None

        # 如果链接是URL，检查它是否在同一域内
        if re.search(HTTP_URL_PATTERN, link):
            # 解析URL并检查域是否相同
            url_obj = urlparse(link)
            if url_obj.netloc == local_domain:
                clean_link = link

        # 如果链接不是URL，请检查它是否是相对链接
        else:
            if link.startswith("/"):
                link = link[1:]
            elif link.startswith("#") or link.startswith("mailto:"):
                continue
            clean_link = "https://" + local_domain + "/" + link

        if clean_link is not None:
            if clean_link.endswith("/"):
                clean_link = clean_link[:-1]
            clean_links.append(clean_link)

    # 返回同一域中的超链接列表
    return list(set(clean_links))


def crawl(url):
    # 解析URL并获取域
    local_domain = urlparse(url).netloc

    # 创建一个队列来存储要抓取的URL
    queue = deque([url])

    # 创建一个集合来存储已经看到的URL（没有重复）
    seen = set([url])

    # 创建一个目录来存储文本文件
    if not os.path.exists("text/"):
            os.mkdir("text/")

    if not os.path.exists("text/"+local_domain+"/"):
            os.mkdir("text/" + local_domain + "/")

    # 创建一个目录来存储csv文件
    if not os.path.exists("processed"):
            os.mkdir("processed")

    # 当队列不为空时，继续
    while queue:

        # 从队列中获取下一个URL
        url = queue.pop()
        print(url)

        # 将文本从url保存到<url>. txt文件
        with open('text/'+local_domain+'/'+url[8:].replace("/", "_") + ".txt", "w", encoding="utf-8") as f:

            # 使用BeautifulSoup从URL获取文本
            soup = BeautifulSoup(requests.get(url).text, "html.parser")

            # 获取文本但删除标签
            text = soup.get_text()

            # 如果爬虫到达需要JavaScript的页面，它将停止爬行
            if ("You need to enable JavaScript to run this app." in text):
                print("Unable to parse page " + url + " due to JavaScript being required")
            
            # 否则，将文本写入文本目录中的文件
            f.write(text)

        # 从URL获取超链接并将它们添加到队列中
        for link in get_domain_hyperlinks(local_domain, url):
            if link not in seen:
                queue.append(link)
                seen.add(link)

crawl(full_url)

https://openai.com/
HTTP Error 403: Forbidden


In [ ]:
import requests
import re
import urllib.request
from bs4 import BeautifulSoup
from collections import deque
from html.parser import HTMLParser
from urllib.parse import urlparse
import os

# 匹配URL的正则表达式模式
HTTP_URL_PATTERN = r'^http[s]*://.+'

# 定义要抓取的网址
domain = "openai.com"
full_url = "https://openai.com/"

# 创建一个类来解析HTML并获取超链接
class HyperlinkParser(HTMLParser):
    def __init__(self):
        super().__init__()
        # 创建一个数组来存储超链接
        self.hyperlinks = []

    # 重写HTMLParser的handle_starttag方法以获取超链接
    def handle_starttag(self, tag, attrs):
        attrs = dict(attrs)

        # 如果标签是锚标签并且具有href属性，则将href属性添加到超链接列表中
        if tag == "a" and "href" in attrs:
            self.hyperlinks.append(attrs["href"])

# 从URL获取超链接的函数
def get_hyperlinks(url):
    
    # 尝试打开URL并阅读超文本标记语言
    try:
        # 打开URL并阅读内容
        with urllib.request.urlopen(url) as response:

            # 如果响应不是html，则返回一个空列表
            if not response.info().get('Content-Type').startswith("text/html"):
                return []
            
            # 解码html
            html = response.read().decode('utf-8')
    except Exception as e:
        print(e)
        return []

    # 创建html解析器，然后解析html以获取超链接
    parser = HyperlinkParser()
    parser.feed(html)

    return parser.hyperlinks

# 从同一域中的URL获取超链接的函数
def get_domain_hyperlinks(local_domain, url):
    clean_links = []
    for link in set(get_hyperlinks(url)):
        clean_link = None

        # 如果链接是URL，检查它是否在同一域内
        if re.search(HTTP_URL_PATTERN, link):
            # 解析URL并检查域是否相同
            url_obj = urlparse(link)
            if url_obj.netloc == local_domain:
                clean_link = link

        # 如果链接不是URL，请检查它是否是相对链接
        else:
            if link.startswith("/"):
                link = link[1:]
            elif link.startswith("#") or link.startswith("mailto:"):
                continue
            clean_link = "https://" + local_domain + "/" + link

        if clean_link is not None:
            if clean_link.endswith("/"):
                clean_link = clean_link[:-1]
            clean_links.append(clean_link)

    # 返回同一域中的超链接列表
    return list(set(clean_links))


def crawl(url):
    # 解析URL并获取域
    local_domain = urlparse(url).netloc

    # 创建一个队列来存储要抓取的URL
    queue = deque([url])

    # 创建一个集合来存储已经看到的URL（没有重复）
    seen = set([url])

    # 创建一个目录来存储文本文件
    if not os.path.exists("text/"):
            os.mkdir("text/")

    if not os.path.exists("text/"+local_domain+"/"):
            os.mkdir("text/" + local_domain + "/")

    # 创建一个目录来存储csv文件
    if not os.path.exists("processed"):
            os.mkdir("processed")

    # 当队列不为空时，继续
    while queue:

        # 从队列中获取下一个URL
        url = queue.pop()
        print(url)

        # 将文本从url保存到<url>. txt文件
        with open('text/'+local_domain+'/'+url[8:].replace("/", "_") + ".txt", "w", encoding="utf-8") as f:

            # 使用BeautifulSoup从URL获取文本
            soup = BeautifulSoup(requests.get(url).text, "html.parser")

            # 获取文本但删除标签
            text = soup.get_text()

            # 如果爬虫到达需要JavaScript的页面，它将停止爬行
            if ("You need to enable JavaScript to run this app." in text):
                print("Unable to parse page " + url + " due to JavaScript being required")
            
            # 否则，将文本写入文本目录中的文件
            f.write(text)

        # 从URL获取超链接并将它们添加到队列中
        for link in get_domain_hyperlinks(local_domain, url):
            if link not in seen:
                queue.append(link)
                seen.add(link)

crawl(full_url)

https://openai.com/
HTTP Error 403: Forbidden


In [ ]:
import requests
import re
import urllib.request
from bs4 import BeautifulSoup
from collections import deque
from html.parser import HTMLParser
from urllib.parse import urlparse
import os

# 匹配URL的正则表达式模式
HTTP_URL_PATTERN = r'^http[s]*://.+'

# 定义要抓取的网址
domain = "openai.com"
full_url = "https://openai.com/"

# 创建一个类来解析HTML并获取超链接
class HyperlinkParser(HTMLParser):
    def __init__(self):
        super().__init__()
        # 创建一个数组来存储超链接
        self.hyperlinks = []

    # 重写HTMLParser的handle_starttag方法以获取超链接
    def handle_starttag(self, tag, attrs):
        attrs = dict(attrs)

        # 如果标签是锚标签并且具有href属性，则将href属性添加到超链接列表中
        if tag == "a" and "href" in attrs:
            self.hyperlinks.append(attrs["href"])

# 从URL获取超链接的函数
def get_hyperlinks(url):
    
    # 尝试打开URL并阅读超文本标记语言
    try:
        # 打开URL并阅读内容
        with urllib.request.urlopen(url) as response:

            # 如果响应不是html，则返回一个空列表
            if not response.info().get('Content-Type').startswith("text/html"):
                return []
            
            # 解码html
            html = response.read().decode('utf-8')
    except Exception as e:
        print(e)
        return []

    # 创建html解析器，然后解析html以获取超链接
    parser = HyperlinkParser()
    parser.feed(html)

    return parser.hyperlinks

# 从同一域中的URL获取超链接的函数
def get_domain_hyperlinks(local_domain, url):
    clean_links = []
    for link in set(get_hyperlinks(url)):
        clean_link = None

        # 如果链接是URL，检查它是否在同一域内
        if re.search(HTTP_URL_PATTERN, link):
            # 解析URL并检查域是否相同
            url_obj = urlparse(link)
            if url_obj.netloc == local_domain:
                clean_link = link

        # 如果链接不是URL，请检查它是否是相对链接
        else:
            if link.startswith("/"):
                link = link[1:]
            elif link.startswith("#") or link.startswith("mailto:"):
                continue
            clean_link = "https://" + local_domain + "/" + link

        if clean_link is not None:
            if clean_link.endswith("/"):
                clean_link = clean_link[:-1]
            clean_links.append(clean_link)

    # 返回同一域中的超链接列表
    return list(set(clean_links))


def crawl(url):
    # 解析URL并获取域
    local_domain = urlparse(url).netloc

    # 创建一个队列来存储要抓取的URL
    queue = deque([url])

    # 创建一个集合来存储已经看到的URL（没有重复）
    seen = set([url])

    # 创建一个目录来存储文本文件
    if not os.path.exists("text/"):
            os.mkdir("text/")

    if not os.path.exists("text/"+local_domain+"/"):
            os.mkdir("text/" + local_domain + "/")

    # 创建一个目录来存储csv文件
    if not os.path.exists("processed"):
            os.mkdir("processed")

    # 当队列不为空时，继续
    while queue:

        # 从队列中获取下一个URL
        url = queue.pop()
        print(url)

        # 将文本从url保存到<url>. txt文件
        with open('text/'+local_domain+'/'+url[8:].replace("/", "_") + ".txt", "w", encoding="utf-8") as f:

            # 使用BeautifulSoup从URL获取文本
            soup = BeautifulSoup(requests.get(url).text, "html.parser")

            # 获取文本但删除标签
            text = soup.get_text()

            # 如果爬虫到达需要JavaScript的页面，它将停止爬行
            if ("You need to enable JavaScript to run this app." in text):
                print("Unable to parse page " + url + " due to JavaScript being required")
            
            # 否则，将文本写入文本目录中的文件
            f.write(text)

        # 从URL获取超链接并将它们添加到队列中
        for link in get_domain_hyperlinks(local_domain, url):
            if link not in seen:
                queue.append(link)
                seen.add(link)

crawl(full_url)

https://openai.com/
HTTP Error 403: Forbidden


In [ ]:
import requests
import re
import urllib.request
from bs4 import BeautifulSoup
from collections import deque
from html.parser import HTMLParser
from urllib.parse import urlparse
import os

# 匹配URL的正则表达式模式
HTTP_URL_PATTERN = r'^http[s]*://.+'

# 定义要抓取的网址
domain = "openai.com"
full_url = "https://openai.com/"

# 创建一个类来解析HTML并获取超链接
class HyperlinkParser(HTMLParser):
    def __init__(self):
        super().__init__()
        # 创建一个数组来存储超链接
        self.hyperlinks = []

    # 重写HTMLParser的handle_starttag方法以获取超链接
    def handle_starttag(self, tag, attrs):
        attrs = dict(attrs)

        # 如果标签是锚标签并且具有href属性，则将href属性添加到超链接列表中
        if tag == "a" and "href" in attrs:
            self.hyperlinks.append(attrs["href"])

# 从URL获取超链接的函数
def get_hyperlinks(url):
    
    # 尝试打开URL并阅读超文本标记语言
    try:
        # 打开URL并阅读内容
        with urllib.request.urlopen(url) as response:

            # 如果响应不是html，则返回一个空列表
            if not response.info().get('Content-Type').startswith("text/html"):
                return []
            
            # 解码html
            html = response.read().decode('utf-8')
    except Exception as e:
        print(e)
        return []

    # 创建html解析器，然后解析html以获取超链接
    parser = HyperlinkParser()
    parser.feed(html)

    return parser.hyperlinks

# 从同一域中的URL获取超链接的函数
def get_domain_hyperlinks(local_domain, url):
    clean_links = []
    for link in set(get_hyperlinks(url)):
        clean_link = None

        # 如果链接是URL，检查它是否在同一域内
        if re.search(HTTP_URL_PATTERN, link):
            # 解析URL并检查域是否相同
            url_obj = urlparse(link)
            if url_obj.netloc == local_domain:
                clean_link = link

        # 如果链接不是URL，请检查它是否是相对链接
        else:
            if link.startswith("/"):
                link = link[1:]
            elif link.startswith("#") or link.startswith("mailto:"):
                continue
            clean_link = "https://" + local_domain + "/" + link

        if clean_link is not None:
            if clean_link.endswith("/"):
                clean_link = clean_link[:-1]
            clean_links.append(clean_link)

    # 返回同一域中的超链接列表
    return list(set(clean_links))


def crawl(url):
    # 解析URL并获取域
    local_domain = urlparse(url).netloc

    # 创建一个队列来存储要抓取的URL
    queue = deque([url])

    # 创建一个集合来存储已经看到的URL（没有重复）
    seen = set([url])

    # 创建一个目录来存储文本文件
    if not os.path.exists("text/"):
            os.mkdir("text/")

    if not os.path.exists("text/"+local_domain+"/"):
            os.mkdir("text/" + local_domain + "/")

    # 创建一个目录来存储csv文件
    if not os.path.exists("processed"):
            os.mkdir("processed")

    # 当队列不为空时，继续
    while queue:

        # 从队列中获取下一个URL
        url = queue.pop()
        print(url)

        # 将文本从url保存到<url>. txt文件
        with open('text/'+local_domain+'/'+url[8:].replace("/", "_") + ".txt", "w", encoding="utf-8") as f:

            # 使用BeautifulSoup从URL获取文本
            soup = BeautifulSoup(requests.get(url).text, "html.parser")

            # 获取文本但删除标签
            text = soup.get_text()

            # 如果爬虫到达需要JavaScript的页面，它将停止爬行
            if ("You need to enable JavaScript to run this app." in text):
                print("Unable to parse page " + url + " due to JavaScript being required")
            
            # 否则，将文本写入文本目录中的文件
            f.write(text)

        # 从URL获取超链接并将它们添加到队列中
        for link in get_domain_hyperlinks(local_domain, url):
            if link not in seen:
                queue.append(link)
                seen.add(link)

crawl(full_url)

https://openai.com/
HTTP Error 403: Forbidden


In [ ]:
import requests
import re
import urllib.request
from bs4 import BeautifulSoup
from collections import deque
from html.parser import HTMLParser
from urllib.parse import urlparse
import os

# 匹配URL的正则表达式模式
HTTP_URL_PATTERN = r'^http[s]*://.+'

# 定义要抓取的网址
domain = "openai.com"
full_url = "https://openai.com/"

# 创建一个类来解析HTML并获取超链接
class HyperlinkParser(HTMLParser):
    def __init__(self):
        super().__init__()
        # 创建一个数组来存储超链接
        self.hyperlinks = []

    # 重写HTMLParser的handle_starttag方法以获取超链接
    def handle_starttag(self, tag, attrs):
        attrs = dict(attrs)

        # 如果标签是锚标签并且具有href属性，则将href属性添加到超链接列表中
        if tag == "a" and "href" in attrs:
            self.hyperlinks.append(attrs["href"])

# 从URL获取超链接的函数
def get_hyperlinks(url):
    
    # 尝试打开URL并阅读超文本标记语言
    try:
        # 打开URL并阅读内容
        with urllib.request.urlopen(url) as response:

            # 如果响应不是html，则返回一个空列表
            if not response.info().get('Content-Type').startswith("text/html"):
                return []
            
            # 解码html
            html = response.read().decode('utf-8')
    except Exception as e:
        print(e)
        return []

    # 创建html解析器，然后解析html以获取超链接
    parser = HyperlinkParser()
    parser.feed(html)

    return parser.hyperlinks

# 从同一域中的URL获取超链接的函数
def get_domain_hyperlinks(local_domain, url):
    clean_links = []
    for link in set(get_hyperlinks(url)):
        clean_link = None

        # 如果链接是URL，检查它是否在同一域内
        if re.search(HTTP_URL_PATTERN, link):
            # 解析URL并检查域是否相同
            url_obj = urlparse(link)
            if url_obj.netloc == local_domain:
                clean_link = link

        # 如果链接不是URL，请检查它是否是相对链接
        else:
            if link.startswith("/"):
                link = link[1:]
            elif link.startswith("#") or link.startswith("mailto:"):
                continue
            clean_link = "https://" + local_domain + "/" + link

        if clean_link is not None:
            if clean_link.endswith("/"):
                clean_link = clean_link[:-1]
            clean_links.append(clean_link)

    # 返回同一域中的超链接列表
    return list(set(clean_links))


def crawl(url):
    # 解析URL并获取域
    local_domain = urlparse(url).netloc

    # 创建一个队列来存储要抓取的URL
    queue = deque([url])

    # 创建一个集合来存储已经看到的URL（没有重复）
    seen = set([url])

    # 创建一个目录来存储文本文件
    if not os.path.exists("text/"):
            os.mkdir("text/")

    if not os.path.exists("text/"+local_domain+"/"):
            os.mkdir("text/" + local_domain + "/")

    # 创建一个目录来存储csv文件
    if not os.path.exists("processed"):
            os.mkdir("processed")

    # 当队列不为空时，继续
    while queue:

        # 从队列中获取下一个URL
        url = queue.pop()
        print(url)

        # 将文本从url保存到<url>. txt文件
        with open('text/'+local_domain+'/'+url[8:].replace("/", "_") + ".txt", "w", encoding="utf-8") as f:

            # 使用BeautifulSoup从URL获取文本
            soup = BeautifulSoup(requests.get(url).text, "html.parser")

            # 获取文本但删除标签
            text = soup.get_text()

            # 如果爬虫到达需要JavaScript的页面，它将停止爬行
            if ("You need to enable JavaScript to run this app." in text):
                print("Unable to parse page " + url + " due to JavaScript being required")
            
            # 否则，将文本写入文本目录中的文件
            f.write(text)

        # 从URL获取超链接并将它们添加到队列中
        for link in get_domain_hyperlinks(local_domain, url):
            if link not in seen:
                queue.append(link)
                seen.add(link)

crawl(full_url)

https://openai.com/
HTTP Error 403: Forbidden


In [ ]:
import requests
import re
import urllib.request
from bs4 import BeautifulSoup
from collections import deque
from html.parser import HTMLParser
from urllib.parse import urlparse
import os

# 匹配URL的正则表达式模式
HTTP_URL_PATTERN = r'^http[s]*://.+'

# 定义要抓取的网址
domain = "openai.com"
full_url = "https://openai.com/"

# 创建一个类来解析HTML并获取超链接
class HyperlinkParser(HTMLParser):
    def __init__(self):
        super().__init__()
        # 创建一个数组来存储超链接
        self.hyperlinks = []

    # 重写HTMLParser的handle_starttag方法以获取超链接
    def handle_starttag(self, tag, attrs):
        attrs = dict(attrs)

        # 如果标签是锚标签并且具有href属性，则将href属性添加到超链接列表中
        if tag == "a" and "href" in attrs:
            self.hyperlinks.append(attrs["href"])

# 从URL获取超链接的函数
def get_hyperlinks(url):
    
    # 尝试打开URL并阅读超文本标记语言
    try:
        # 打开URL并阅读内容
        with urllib.request.urlopen(url) as response:

            # 如果响应不是html，则返回一个空列表
            if not response.info().get('Content-Type').startswith("text/html"):
                return []
            
            # 解码html
            html = response.read().decode('utf-8')
    except Exception as e:
        print(e)
        return []

    # 创建html解析器，然后解析html以获取超链接
    parser = HyperlinkParser()
    parser.feed(html)

    return parser.hyperlinks

# 从同一域中的URL获取超链接的函数
def get_domain_hyperlinks(local_domain, url):
    clean_links = []
    for link in set(get_hyperlinks(url)):
        clean_link = None

        # 如果链接是URL，检查它是否在同一域内
        if re.search(HTTP_URL_PATTERN, link):
            # 解析URL并检查域是否相同
            url_obj = urlparse(link)
            if url_obj.netloc == local_domain:
                clean_link = link

        # 如果链接不是URL，请检查它是否是相对链接
        else:
            if link.startswith("/"):
                link = link[1:]
            elif link.startswith("#") or link.startswith("mailto:"):
                continue
            clean_link = "https://" + local_domain + "/" + link

        if clean_link is not None:
            if clean_link.endswith("/"):
                clean_link = clean_link[:-1]
            clean_links.append(clean_link)

    # 返回同一域中的超链接列表
    return list(set(clean_links))


def crawl(url):
    # 解析URL并获取域
    local_domain = urlparse(url).netloc

    # 创建一个队列来存储要抓取的URL
    queue = deque([url])

    # 创建一个集合来存储已经看到的URL（没有重复）
    seen = set([url])

    # 创建一个目录来存储文本文件
    if not os.path.exists("text/"):
            os.mkdir("text/")

    if not os.path.exists("text/"+local_domain+"/"):
            os.mkdir("text/" + local_domain + "/")

    # 创建一个目录来存储csv文件
    if not os.path.exists("processed"):
            os.mkdir("processed")

    # 当队列不为空时，继续
    while queue:

        # 从队列中获取下一个URL
        url = queue.pop()
        print(url)

        # 将文本从url保存到<url>. txt文件
        with open('text/'+local_domain+'/'+url[8:].replace("/", "_") + ".txt", "w", encoding="utf-8") as f:

            # 使用BeautifulSoup从URL获取文本
            soup = BeautifulSoup(requests.get(url).text, "html.parser")

            # 获取文本但删除标签
            text = soup.get_text()

            # 如果爬虫到达需要JavaScript的页面，它将停止爬行
            if ("You need to enable JavaScript to run this app." in text):
                print("Unable to parse page " + url + " due to JavaScript being required")
            
            # 否则，将文本写入文本目录中的文件
            f.write(text)

        # 从URL获取超链接并将它们添加到队列中
        for link in get_domain_hyperlinks(local_domain, url):
            if link not in seen:
                queue.append(link)
                seen.add(link)

crawl(full_url)

https://openai.com/
HTTP Error 403: Forbidden


In [ ]:
import requests
import re
import urllib.request
from bs4 import BeautifulSoup
from collections import deque
from html.parser import HTMLParser
from urllib.parse import urlparse
import os

# 匹配URL的正则表达式模式
HTTP_URL_PATTERN = r'^http[s]*://.+'

# 定义要抓取的网址
domain = "openai.com"
full_url = "https://openai.com/"

# 创建一个类来解析HTML并获取超链接
class HyperlinkParser(HTMLParser):
    def __init__(self):
        super().__init__()
        # 创建一个数组来存储超链接
        self.hyperlinks = []

    # 重写HTMLParser的handle_starttag方法以获取超链接
    def handle_starttag(self, tag, attrs):
        attrs = dict(attrs)

        # 如果标签是锚标签并且具有href属性，则将href属性添加到超链接列表中
        if tag == "a" and "href" in attrs:
            self.hyperlinks.append(attrs["href"])

# 从URL获取超链接的函数
def get_hyperlinks(url):
    
    # 尝试打开URL并阅读超文本标记语言
    try:
        # 打开URL并阅读内容
        with urllib.request.urlopen(url) as response:

            # 如果响应不是html，则返回一个空列表
            if not response.info().get('Content-Type').startswith("text/html"):
                return []
            
            # 解码html
            html = response.read().decode('utf-8')
    except Exception as e:
        print(e)
        return []

    # 创建html解析器，然后解析html以获取超链接
    parser = HyperlinkParser()
    parser.feed(html)

    return parser.hyperlinks

# 从同一域中的URL获取超链接的函数
def get_domain_hyperlinks(local_domain, url):
    clean_links = []
    for link in set(get_hyperlinks(url)):
        clean_link = None

        # 如果链接是URL，检查它是否在同一域内
        if re.search(HTTP_URL_PATTERN, link):
            # 解析URL并检查域是否相同
            url_obj = urlparse(link)
            if url_obj.netloc == local_domain:
                clean_link = link

        # 如果链接不是URL，请检查它是否是相对链接
        else:
            if link.startswith("/"):
                link = link[1:]
            elif link.startswith("#") or link.startswith("mailto:"):
                continue
            clean_link = "https://" + local_domain + "/" + link

        if clean_link is not None:
            if clean_link.endswith("/"):
                clean_link = clean_link[:-1]
            clean_links.append(clean_link)

    # 返回同一域中的超链接列表
    return list(set(clean_links))


def crawl(url):
    # 解析URL并获取域
    local_domain = urlparse(url).netloc

    # 创建一个队列来存储要抓取的URL
    queue = deque([url])

    # 创建一个集合来存储已经看到的URL（没有重复）
    seen = set([url])

    # 创建一个目录来存储文本文件
    if not os.path.exists("text/"):
            os.mkdir("text/")

    if not os.path.exists("text/"+local_domain+"/"):
            os.mkdir("text/" + local_domain + "/")

    # 创建一个目录来存储csv文件
    if not os.path.exists("processed"):
            os.mkdir("processed")

    # 当队列不为空时，继续
    while queue:

        # 从队列中获取下一个URL
        url = queue.pop()
        print(url)

        # 将文本从url保存到<url>. txt文件
        with open('text/'+local_domain+'/'+url[8:].replace("/", "_") + ".txt", "w", encoding="utf-8") as f:

            # 使用BeautifulSoup从URL获取文本
            soup = BeautifulSoup(requests.get(url).text, "html.parser")

            # 获取文本但删除标签
            text = soup.get_text()

            # 如果爬虫到达需要JavaScript的页面，它将停止爬行
            if ("You need to enable JavaScript to run this app." in text):
                print("Unable to parse page " + url + " due to JavaScript being required")
            
            # 否则，将文本写入文本目录中的文件
            f.write(text)

        # 从URL获取超链接并将它们添加到队列中
        for link in get_domain_hyperlinks(local_domain, url):
            if link not in seen:
                queue.append(link)
                seen.add(link)

crawl(full_url)

https://openai.com/
HTTP Error 403: Forbidden


In [ ]:
import requests
import re
import urllib.request
from bs4 import BeautifulSoup
from collections import deque
from html.parser import HTMLParser
from urllib.parse import urlparse
import os

# 匹配URL的正则表达式模式
HTTP_URL_PATTERN = r'^http[s]*://.+'

# 定义要抓取的网址
domain = "openai.com"
full_url = "https://openai.com/"

# 创建一个类来解析HTML并获取超链接
class HyperlinkParser(HTMLParser):
    def __init__(self):
        super().__init__()
        # 创建一个数组来存储超链接
        self.hyperlinks = []

    # 重写HTMLParser的handle_starttag方法以获取超链接
    def handle_starttag(self, tag, attrs):
        attrs = dict(attrs)

        # 如果标签是锚标签并且具有href属性，则将href属性添加到超链接列表中
        if tag == "a" and "href" in attrs:
            self.hyperlinks.append(attrs["href"])

# 从URL获取超链接的函数
def get_hyperlinks(url):
    
    # 尝试打开URL并阅读超文本标记语言
    try:
        # 打开URL并阅读内容
        with urllib.request.urlopen(url) as response:

            # 如果响应不是html，则返回一个空列表
            if not response.info().get('Content-Type').startswith("text/html"):
                return []
            
            # 解码html
            html = response.read().decode('utf-8')
    except Exception as e:
        print(e)
        return []

    # 创建html解析器，然后解析html以获取超链接
    parser = HyperlinkParser()
    parser.feed(html)

    return parser.hyperlinks

# 从同一域中的URL获取超链接的函数
def get_domain_hyperlinks(local_domain, url):
    clean_links = []
    for link in set(get_hyperlinks(url)):
        clean_link = None

        # 如果链接是URL，检查它是否在同一域内
        if re.search(HTTP_URL_PATTERN, link):
            # 解析URL并检查域是否相同
            url_obj = urlparse(link)
            if url_obj.netloc == local_domain:
                clean_link = link

        # 如果链接不是URL，请检查它是否是相对链接
        else:
            if link.startswith("/"):
                link = link[1:]
            elif link.startswith("#") or link.startswith("mailto:"):
                continue
            clean_link = "https://" + local_domain + "/" + link

        if clean_link is not None:
            if clean_link.endswith("/"):
                clean_link = clean_link[:-1]
            clean_links.append(clean_link)

    # 返回同一域中的超链接列表
    return list(set(clean_links))


def crawl(url):
    # 解析URL并获取域
    local_domain = urlparse(url).netloc

    # 创建一个队列来存储要抓取的URL
    queue = deque([url])

    # 创建一个集合来存储已经看到的URL（没有重复）
    seen = set([url])

    # 创建一个目录来存储文本文件
    if not os.path.exists("text/"):
            os.mkdir("text/")

    if not os.path.exists("text/"+local_domain+"/"):
            os.mkdir("text/" + local_domain + "/")

    # 创建一个目录来存储csv文件
    if not os.path.exists("processed"):
            os.mkdir("processed")

    # 当队列不为空时，继续
    while queue:

        # 从队列中获取下一个URL
        url = queue.pop()
        print(url)

        # 将文本从url保存到<url>. txt文件
        with open('text/'+local_domain+'/'+url[8:].replace("/", "_") + ".txt", "w", encoding="utf-8") as f:

            # 使用BeautifulSoup从URL获取文本
            soup = BeautifulSoup(requests.get(url).text, "html.parser")

            # 获取文本但删除标签
            text = soup.get_text()

            # 如果爬虫到达需要JavaScript的页面，它将停止爬行
            if ("You need to enable JavaScript to run this app." in text):
                print("Unable to parse page " + url + " due to JavaScript being required")
            
            # 否则，将文本写入文本目录中的文件
            f.write(text)

        # 从URL获取超链接并将它们添加到队列中
        for link in get_domain_hyperlinks(local_domain, url):
            if link not in seen:
                queue.append(link)
                seen.add(link)

crawl(full_url)

https://openai.com/
HTTP Error 403: Forbidden


In [ ]:
import requests
import re
import urllib.request
from bs4 import BeautifulSoup
from collections import deque
from html.parser import HTMLParser
from urllib.parse import urlparse
import os

# 匹配URL的正则表达式模式
HTTP_URL_PATTERN = r'^http[s]*://.+'

# 定义要抓取的网址
domain = "openai.com"
full_url = "https://openai.com/"

# 创建一个类来解析HTML并获取超链接
class HyperlinkParser(HTMLParser):
    def __init__(self):
        super().__init__()
        # 创建一个数组来存储超链接
        self.hyperlinks = []

    # 重写HTMLParser的handle_starttag方法以获取超链接
    def handle_starttag(self, tag, attrs):
        attrs = dict(attrs)

        # 如果标签是锚标签并且具有href属性，则将href属性添加到超链接列表中
        if tag == "a" and "href" in attrs:
            self.hyperlinks.append(attrs["href"])

# 从URL获取超链接的函数
def get_hyperlinks(url):
    
    # 尝试打开URL并阅读超文本标记语言
    try:
        # 打开URL并阅读内容
        with urllib.request.urlopen(url) as response:

            # 如果响应不是html，则返回一个空列表
            if not response.info().get('Content-Type').startswith("text/html"):
                return []
            
            # 解码html
            html = response.read().decode('utf-8')
    except Exception as e:
        print(e)
        return []

    # 创建html解析器，然后解析html以获取超链接
    parser = HyperlinkParser()
    parser.feed(html)

    return parser.hyperlinks

# 从同一域中的URL获取超链接的函数
def get_domain_hyperlinks(local_domain, url):
    clean_links = []
    for link in set(get_hyperlinks(url)):
        clean_link = None

        # 如果链接是URL，检查它是否在同一域内
        if re.search(HTTP_URL_PATTERN, link):
            # 解析URL并检查域是否相同
            url_obj = urlparse(link)
            if url_obj.netloc == local_domain:
                clean_link = link

        # 如果链接不是URL，请检查它是否是相对链接
        else:
            if link.startswith("/"):
                link = link[1:]
            elif link.startswith("#") or link.startswith("mailto:"):
                continue
            clean_link = "https://" + local_domain + "/" + link

        if clean_link is not None:
            if clean_link.endswith("/"):
                clean_link = clean_link[:-1]
            clean_links.append(clean_link)

    # 返回同一域中的超链接列表
    return list(set(clean_links))


def crawl(url):
    # 解析URL并获取域
    local_domain = urlparse(url).netloc

    # 创建一个队列来存储要抓取的URL
    queue = deque([url])

    # 创建一个集合来存储已经看到的URL（没有重复）
    seen = set([url])

    # 创建一个目录来存储文本文件
    if not os.path.exists("text/"):
            os.mkdir("text/")

    if not os.path.exists("text/"+local_domain+"/"):
            os.mkdir("text/" + local_domain + "/")

    # 创建一个目录来存储csv文件
    if not os.path.exists("processed"):
            os.mkdir("processed")

    # 当队列不为空时，继续
    while queue:

        # 从队列中获取下一个URL
        url = queue.pop()
        print(url)

        # 将文本从url保存到<url>. txt文件
        with open('text/'+local_domain+'/'+url[8:].replace("/", "_") + ".txt", "w", encoding="utf-8") as f:

            # 使用BeautifulSoup从URL获取文本
            soup = BeautifulSoup(requests.get(url).text, "html.parser")

            # 获取文本但删除标签
            text = soup.get_text()

            # 如果爬虫到达需要JavaScript的页面，它将停止爬行
            if ("You need to enable JavaScript to run this app." in text):
                print("Unable to parse page " + url + " due to JavaScript being required")
            
            # 否则，将文本写入文本目录中的文件
            f.write(text)

        # 从URL获取超链接并将它们添加到队列中
        for link in get_domain_hyperlinks(local_domain, url):
            if link not in seen:
                queue.append(link)
                seen.add(link)

crawl(full_url)

https://openai.com/
HTTP Error 403: Forbidden


In [ ]:
import requests
import re
import urllib.request
from bs4 import BeautifulSoup
from collections import deque
from html.parser import HTMLParser
from urllib.parse import urlparse
import os

# 匹配URL的正则表达式模式
HTTP_URL_PATTERN = r'^http[s]*://.+'

# 定义要抓取的网址
domain = "openai.com"
full_url = "https://openai.com/"

# 创建一个类来解析HTML并获取超链接
class HyperlinkParser(HTMLParser):
    def __init__(self):
        super().__init__()
        # 创建一个数组来存储超链接
        self.hyperlinks = []

    # 重写HTMLParser的handle_starttag方法以获取超链接
    def handle_starttag(self, tag, attrs):
        attrs = dict(attrs)

        # 如果标签是锚标签并且具有href属性，则将href属性添加到超链接列表中
        if tag == "a" and "href" in attrs:
            self.hyperlinks.append(attrs["href"])

# 从URL获取超链接的函数
def get_hyperlinks(url):
    
    # 尝试打开URL并阅读超文本标记语言
    try:
        # 打开URL并阅读内容
        with urllib.request.urlopen(url) as response:

            # 如果响应不是html，则返回一个空列表
            if not response.info().get('Content-Type').startswith("text/html"):
                return []
            
            # 解码html
            html = response.read().decode('utf-8')
    except Exception as e:
        print(e)
        return []

    # 创建html解析器，然后解析html以获取超链接
    parser = HyperlinkParser()
    parser.feed(html)

    return parser.hyperlinks

# 从同一域中的URL获取超链接的函数
def get_domain_hyperlinks(local_domain, url):
    clean_links = []
    for link in set(get_hyperlinks(url)):
        clean_link = None

        # 如果链接是URL，检查它是否在同一域内
        if re.search(HTTP_URL_PATTERN, link):
            # 解析URL并检查域是否相同
            url_obj = urlparse(link)
            if url_obj.netloc == local_domain:
                clean_link = link

        # 如果链接不是URL，请检查它是否是相对链接
        else:
            if link.startswith("/"):
                link = link[1:]
            elif link.startswith("#") or link.startswith("mailto:"):
                continue
            clean_link = "https://" + local_domain + "/" + link

        if clean_link is not None:
            if clean_link.endswith("/"):
                clean_link = clean_link[:-1]
            clean_links.append(clean_link)

    # 返回同一域中的超链接列表
    return list(set(clean_links))


def crawl(url):
    # 解析URL并获取域
    local_domain = urlparse(url).netloc

    # 创建一个队列来存储要抓取的URL
    queue = deque([url])

    # 创建一个集合来存储已经看到的URL（没有重复）
    seen = set([url])

    # 创建一个目录来存储文本文件
    if not os.path.exists("text/"):
            os.mkdir("text/")

    if not os.path.exists("text/"+local_domain+"/"):
            os.mkdir("text/" + local_domain + "/")

    # 创建一个目录来存储csv文件
    if not os.path.exists("processed"):
            os.mkdir("processed")

    # 当队列不为空时，继续
    while queue:

        # 从队列中获取下一个URL
        url = queue.pop()
        print(url)

        # 将文本从url保存到<url>. txt文件
        with open('text/'+local_domain+'/'+url[8:].replace("/", "_") + ".txt", "w", encoding="utf-8") as f:

            # 使用BeautifulSoup从URL获取文本
            soup = BeautifulSoup(requests.get(url).text, "html.parser")

            # 获取文本但删除标签
            text = soup.get_text()

            # 如果爬虫到达需要JavaScript的页面，它将停止爬行
            if ("You need to enable JavaScript to run this app." in text):
                print("Unable to parse page " + url + " due to JavaScript being required")
            
            # 否则，将文本写入文本目录中的文件
            f.write(text)

        # 从URL获取超链接并将它们添加到队列中
        for link in get_domain_hyperlinks(local_domain, url):
            if link not in seen:
                queue.append(link)
                seen.add(link)

crawl(full_url)

https://openai.com/
HTTP Error 403: Forbidden


In [ ]:
def remove_newlines(serie):
    """
    移除字符串中的换行符。
    
    参数:
    serie: pandas序列，包含需要处理的字符串。
    
    返回:
    处理后的pandas序列，其中的换行符已被替换为单个空格。
    """
    # 替换UNIX风格的换行符
    serie = serie.str.replace('\n', ' ')
    # 替换Windows风格的换行符
    serie = serie.str.replace('\\n', ' ')
    # 压缩连续的两个空格为一个，以避免不必要的空格
    serie = serie.str.replace('  ', ' ')
    # 再次压缩，确保所有的连续空格都被处理，这是对前一步的保险措施
    serie = serie.str.replace('  ', ' ')
    return serie

In [ ]:
import pandas as pd

# 创建一个列表来存储文本文件
texts=[]

# 遍历指定域名下的所有文本文件
for file in os.listdir("text/" + domain + "/"):
    # 以只读模式打开文件
    with open("text/" + domain + "/" + file, "r") as f:
        # 读取文件内容
        text = f.read()
        # 处理文件名，去除前导字符和尾部字符，并替换特定字符为空格
        # 这是为了规范化文件名，使其更适合后续处理
        processed_file_name = file[11:-4].replace('-',' ').replace('_', ' ').replace('#update','')
        # 将文件名和内容作为一个元组添加到列表中
        texts.append((processed_file_name, text))

# 使用列表创建一个DataFrame，指定列名为'fname'和'text'
df = pd.DataFrame(texts, columns = ['fname', 'text'])

# 合并文件名和内容，去除换行符，并更新文本列
df['text'] = df.fname + ". " + remove_newlines(df.text)
# 将处理后的数据保存到CSV文件中
df.to_csv('processed/scraped.csv')
# 显示DataFrame的前5行，用于检查数据是否正确处理
df.head()

In [ ]:
import tiktoken

# 加载设计用于ada-002模型的cl100k_base标记器
tokenizer = tiktoken.get_encoding("cl100k_base")

df = pd.read_csv('processed/scraped.csv', index_col=0)
df.columns = ['title', 'text']

# 标记文本并将标记数保存到新列
df['n_tokens'] = df.text.apply(lambda x: len(tokenizer.encode(x)))

# 使用直方图可视化每行标记数量的分布
df.n_tokens.hist()

In [ ]:
max_tokens = 500

# 将文本拆分为最大标记数的块的函数
def split_into_many(text, max_tokens = max_tokens):

    sentences = text.split('. ')

    # 获取每个句子的标记数
    n_tokens = [len(tokenizer.encode(" " + sentence)) for sentence in sentences]
    
    chunks = []
    tokens_so_far = 0
    chunk = []

    # 循环遍历连接在元组中的句子和标记
    for sentence, token in zip(sentences, n_tokens):

        # 如果到目前为止的标记数加上当前句子中的标记数更大
        # 超过令牌的最大数量，然后将块添加到块列表并重置到目前为止的块和令牌
        if tokens_so_far + token > max_tokens:
            chunks.append(". ".join(chunk) + ".")
            chunk = []
            tokens_so_far = 0

        #如果当前句子中的标记数大于最大标记数，则转到下一句
        if token > max_tokens:
            continue

        #否则，将句子添加到块中，并将标记数添加到总数中
        chunk.append(sentence)
        tokens_so_far += token + 1

    #将最后一个块添加到块列表中
    if chunk:
        chunks.append(". ".join(chunk) + ".")

    return chunks
    

shortened = []

#循环通过数据帧
for row in df.iterrows():

    if row[1]['text'] is None:
        continue

    #如果标记数大于最大标记数，则将文本拆分为块
    if row[1]['n_tokens'] > max_tokens:
        shortened += split_into_many(row[1]['text'])
    
    #否则，将文本添加到缩短文本列表中
    else:
        shortened.append( row[1]['text'] )

In [ ]:
df = pd.DataFrame(shortened, columns = ['text'])
df['n_tokens'] = df.text.apply(lambda x: len(tokenizer.encode(x)))
df.n_tokens.hist()

In [ ]:
import openai

df['embeddings'] = df.text.apply(lambda x: openai.Embedding.create(input=x, engine='text-embedding-ada-002')['data'][0]['embedding'])
df.to_csv('processed/embeddings.csv')
df.head()

In [ ]:
import pandas as pd
import numpy as np
from ast import literal_eval
from openai.embeddings_utils import distances_from_embeddings, cosine_similarity

df=pd.read_csv('processed/embeddings.csv', index_col=0)
df['embeddings'] = df['embeddings'].apply(literal_eval).apply(np.array)

df.head()

In [ ]:
def create_context(
    question, df, max_len=1800, size="ada"
):
    """
    Create a context for a question by finding the most similar context from the dataframe
    """

    #获取问题的嵌入
    q_embeddings = openai.Embedding.create(input=question, engine='text-embedding-ada-002')['data'][0]['embedding']

    #获取与嵌入的距离
    df['distances'] = distances_from_embeddings(q_embeddings, df['embeddings'].values, distance_metric='cosine')


    returns = []
    cur_len = 0

    #按距离排序并将文本添加到上下文中，直到上下文太长
    for i, row in df.sort_values('distances', ascending=True).iterrows():
        
        #将文本的长度添加到当前长度
        cur_len += row['n_tokens'] + 4
        
        #如果上下文太长，中断
        if cur_len > max_len:
            break
        
        #否则将其添加到正在返回的文本中
        returns.append(row["text"])

    #返回上下文
    return "\n\n###\n\n".join(returns)

def answer_question(
    df,
    model="text-davinci-003",
    question="Am I allowed to publish model outputs to Twitter, without a human review?",
    max_len=1800,
    size="ada",
    debug=False,
    max_tokens=150,
    stop_sequence=None
):
    """
    Answer a question based on the most similar context from the dataframe texts
    """
    context = create_context(
        question,
        df,
        max_len=max_len,
        size=size,
    )
    #如果调试，打印原始模型响应
    if debug:
        print("Context:\n" + context)
        print("\n\n")

    try:
        #使用问题和上下文创建完成
        response = openai.Completion.create(
            prompt=f"Answer the question based on the context below, and if the question can't be answered based on the context, say \"I don't know\"\n\nContext: {context}\n\n---\n\nQuestion: {question}\nAnswer:",
            temperature=0,
            max_tokens=max_tokens,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0,
            stop=stop_sequence,
            model=model,
        )
        return response["choices"][0]["text"].strip()
    except Exception as e:
        print(e)
        return ""

In [ ]:
answer_question(df, question="What day is it?", debug=False)

In [ ]:
answer_question(df, question="What is our newest embeddings model?")